In [216]:
### Create interactive map of residents living within 400 - 1600 m (+ 250 m because of the grid resolution) walking distance from 
### metro and train station in Finland's capital region with a value slider. Data for the map is loaded from open data services of 
### Maanmittauslaitos (MML) and Helsinki Region Environmental Services Authority (HSY).

### IMPORT DATA ###

# Import modules
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geopandas.tools import geocode
import numpy as np
from pyproj import CRS
import requests
import geojson
import matplotlib.pyplot as plt
from shapely.ops import cascaded_union
import mapclassify
import contextily as ctx
from mpl_toolkits.axes_grid1 import make_axes_locatable

import folium
import folium.plugins
import branca
import branca.colormap as cm

## Read shape file containing the capital region as polygons into variable 'grid' (Data from https://tiedostopalvelu.maanmittauslaitos.fi/tp/kartta)
# File path
fp_grid = "data/pkseutu.shp"

# Read in data
grid = gpd.read_file(fp_grid)

# Check if crs is correct and set crs to ETRS89 / TM35FIN if the crs is not defined correctly
if (grid.crs != "epsg:3067"):    
    grid = grid.set_crs(epsg=3067)
# Reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (grid.crs != "epsg:4326"):    
    grid = grid.to_crs(epsg=4326)

# Combine polygons of each city to form one polygon of the whole capial region
grid['constant'] = 0
boundary = grid.dissolve(by='constant')

# Check the data
#print(grid.head())
#print(grid.crs)
#print(boundary)

In [217]:
## Read population grid data for 2018 into a variable `pop`. 

# Specify the url for web feature service
url = 'https://kartta.hsy.fi/geoserver/wfs'

# Specify parameters (read data in json format).
params = dict(service='WFS',
              version='2.0.0',
              request='GetFeature',
              typeName='asuminen_ja_maankaytto:Vaestotietoruudukko_2018',
              outputFormat='json')

# Fetch data from WFS using requests
r = requests.get(url, params=params)

# Create GeoDataFrame from geojson
pop = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

# Clean out unnecessary columns
pop = pop[["asukkaita", "geometry"]]

# Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
if (pop.crs == None):    
    pop = pop.set_crs(epsg=3879)
if (pop.crs != "epsg:4326"):    
    pop = pop.to_crs(epsg=4326)

# Check the data
#print(pop.head())
#print(pop.crs)

In [219]:
## Read buffer polygons that describe 400 m, 800 m, 1200 m and 1600 m accessibilities via pedestrian and bicycle ways from metro and 
## train stations 

# Save wanted buffer sizes in a list which is used in loading the data
dists = ['400', '800', '1200', '1600']

# Create an empty geopandas GeoDataFrame for the data
buffs = gpd.GeoDataFrame()

# Iterate through wanted buffer distance list
for dist in dists:

    # Specify the url for web feature service and typeName of the data layer
    url_buff = 'https://kartta.hsy.fi/geoserver/wfs'
    type_name = dist + 'm_verkostobufferi'

    # Specify parameters (read data in json format).
    params_buff = dict(service='WFS',
                  version='2.0.0',
                  request='GetFeature',
                  typeName=type_name,
                  outputFormat='json')

    # Fetch data from WFS using requests
    r = requests.get(url_buff, params=params_buff)

    # Create GeoDataFrame from geojson
    buff = gpd.GeoDataFrame.from_features(geojson.loads(r.content))

    # Clean out unnecessary columns
    buff = buff[["asema", "geometry"]]

    # Set crs to ETRS89 / GK25FIN and reproject to WGS 84 / Pseudo-Mercator if the crs is not defined correctly
    if (buff.crs == None):    
        buff = buff.set_crs(epsg=3879)
    if (buff.crs != "epsg:4326"):    
        buff = buff.to_crs(epsg=4326)

    # Clip out stations that are located outside the capital region
    clip_mask = buff.within(boundary.at[0,'geometry'])
    buff = buff.loc[clip_mask]
    
    # Create column which indicates buffer distance for the slider
    buff['dist'] = dist

    # Check the data
    #print(buff.head(1))
    #print(len(buff))

    # Add the data to combined GeoDataFrame
    buffs = buffs.append(buff)

# Remove manually one station that wasn't clipped out 
buffs = buffs[buffs.asema != 'Mankki']
    
# Check the data
#buffs.head()

In [220]:
### PROCESS DATA ###

# Create new column to 'buffs' where total resident amounts within each buffer areas are stored 
buffs["residents_sum"] = None

# Create a spatial join between grid layer and buffer layer. "Intersects" option used here to include all grid cells which 
# touch the buffer area (NOTE that with this choice the accuracy of the buffers is lost due to the grid resolution)
pop_combined = gpd.sjoin(pop, buffs, how="left", op="intersects")

# Group the data by both train and metro station names AND distance classes
groupedA = pop_combined.groupby(['asema','dist'])

# Check the data
#groupedA.head()

In [221]:
# Store sum of residents living approximately 400 m, 800 m, 1200 m and 1600 m from station to column "sum" 
# (the distance doesn't stay constant in performed analysis but accurate enough for this visualization)
for name, group in groupedA:
    buffs.loc[(buffs["asema"]==name[0]) & (buffs['dist']==name[1]),'residents_sum'] = group["asukkaita"].agg("sum")
    
    
## Convert the buffer polygons to points (location set as centroids of 400 m buffers, approximate of the station locations)
point_data = buffs
point_data = point_data.reset_index()

# Replace NoData in residents_sum column with 0
point_data["residents_sum"] = point_data["residents_sum"].replace(to_replace=np.nan, value=0)

# Group the data by only train and metro station names
groupedB = point_data.groupby('asema')

# Convert to points based on centroids
for name, group in groupedB:
    point_data.loc[point_data["asema"]==name,'geometry'] = group['geometry'].centroid # NOTE: raises an warning which ignored in this instance
    
# Reorganize the column order
point_data = point_data[["geometry","asema","residents_sum", "dist"]]

# Check the data
#point_data.head()

<ipython-input-221-bc21eef78e55>:19: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  point_data.loc[point_data["asema"]==name,'geometry'] = group['geometry'].centroid


In [215]:
### PREPARE AND DIVIDE DATA

# Divide data from each buffer distances into separate GeoDataFrames
buff400 = point_data.loc[point_data['dist']=='400']
buff800 = point_data.loc[point_data['dist']=='800']
buff1200 = point_data.loc[point_data['dist']=='1200']
buff1600 = point_data.loc[point_data['dist']=='1600']

# Sort rows by station name
buff400 = buff400.sort_values(by=['asema'])
buff800 = buff800.sort_values(by=['asema'])
buff1200 = buff1200.sort_values(by=['asema'])
buff1600 = buff1600.sort_values(by=['asema'])

# Get x and y coordinates for each point in buff400
buff400["x"] = buff400["geometry"].apply(lambda geom: geom.x)
buff400["y"] = buff400["geometry"].apply(lambda geom: geom.y)

# Set same station coordinates for each buffer distances
buff800['x'] = buff400['x'].values
buff800['y'] = buff400['y'].values

buff1200['x'] = buff400['x'].values
buff1200['y'] = buff400['y'].values

buff1600['x'] = buff400['x'].values
buff1600['y'] = buff400['y'].values

In [209]:
### PLOT DATA

# Plot the basemap
m = folium.Map(location=[60.24026, 24.96179], tiles = 'cartodbpositron',
                zoom_start=10, control_scale=True, prefer_canvas=True)

# Create colormap
colormap = cm.LinearColormap(colors=['yellow','red'], index=[0,85000],vmin=0,vmax=max(point_data['residents_sum']),
                            caption='Residents living within defined walking distance from metro or train station')

# Define tool which creates point markers from input GeoDataSeries
def station_style(station):
    return folium.CircleMarker(
        radius=3,
        location=(station.y, station.x), 
        color=colormap(station.residents_sum),
        tooltip=station.asema + ",\n" + str(station.residents_sum) + " residents",
        fill=True,
        smooth_factor=1.0,
        fill_opacity=.8)

# Create empty FeatureGroup for each buffer
stations400 = folium.FeatureGroup(name='Residents within 400 m from metro and train stations')
stations800 = folium.FeatureGroup(name='Residents within 800 m from metro and train stations')
stations1200 = folium.FeatureGroup(name='Residents within 1200 m from metro and train stations')
stations1600 = folium.FeatureGroup(name='Residents within 1600 m from metro and train stations')

# Add data of each buffer to separate FeatureGroups 
for station in buff400.itertuples():
    station_style(station).add_to(stations400)
    
for station in buff800.itertuples():
    station_style(station).add_to(stations800)
    
for station in buff1200.itertuples():
    station_style(station).add_to(stations1200)
    
for station in buff1600.itertuples():
    station_style(station).add_to(stations1600)

# Add point layers to map
stations400.add_to(m)
stations800.add_to(m)
stations1200.add_to(m)
stations1600.add_to(m)

# Create and add a layer control object 
folium.LayerControl('topleft').add_to(m)

# Add legend 
colormap.add_to(m)

# Save the map
outfp = "choropleth_map.html"
m.save(outfp)

m